## 1. Discretize of beam elements
Improving Timoshenko's Model Represented by Euler Angles.


### 1.1. Representation using quaternions
Since NSK uses quaternions to handle posture, we will model the quaternion representation in this program as well. Although there is an idea to convert Euler angles and quaternions for each calculation, we will use the direct quaternion method this time because it seems to be computationally expensive and there are already previous studies.

#### 1.1.1. Why quaternions?
Light computational cost. Gimbal lock does not occur.

|                 | Parameters | Memory | Handling | Notes                                                                   |
|:---------------:|:----------:|:------:|:--------:|:-----------------------------------------------------------------------:|
| Quaternion      | 4          | 0.5    | 1        | Has a good balance of abilities                                         |
| Eular           | 3          | 1      | 0        | The gradient per unit attitude change differs depending on the attitude |
| Rotation Matrix | 9          | 0      | 0.5      |                                                                         |

In [106]:
import numpy as np  # The next command is required: "pip install -U numpy"
import quaternion   # The next command is required: "pip install numpy-quaternion"


#### Simple Example

Make $q_0$ which means $60^o(= \frac{\pi}{3})$ degree rotation around the $z(=\begin{bmatrix} 0\\0\\1\end{bmatrix})$ axis.


In [107]:
q0 = quaternion.from_rotation_vector(np.array([0, 0, 1]) * np.pi / 3)
print(q0)

quaternion(0.866025403784439, 0, 0, 0.5)


$$
q_0 = \begin{bmatrix} \cos \frac{\theta}{2}\\ \sin \frac{\theta}{2} \begin{bmatrix} x\\y\\z \end{bmatrix} \end{bmatrix} = \begin{bmatrix} \cos \frac{60^0}{2}\\ 0 \\ 0 \\ \sin \frac{60^0}{2} \end{bmatrix} = \begin{bmatrix} \frac{\sqrt{3}}{2}\\ 0 \\ 0 \\ \frac{1}{2} \end{bmatrix}
$$

If we use this quaternion to calculate the rotation, we get this result.


In [108]:
print('Rotated x: ' + str(quaternion.rotate_vectors(q0, np.array([1, 0, 0]))))
print('Rotated y: ' + str(quaternion.rotate_vectors(q0, np.array([0, 1, 0]))))
print('Rotated z: ' + str(quaternion.rotate_vectors(q0, np.array([0, 0, 1]))))


Rotated x: [0.5       0.8660254 0.       ]
Rotated y: [-0.8660254  0.5        0.       ]
Rotated z: [0. 0. 1.]


$$
q_0 \otimes \begin{bmatrix} 1\\0\\0\end{bmatrix}=\begin{bmatrix} \frac{1}{2}\\\frac{\sqrt{3}}{2}\\0\end{bmatrix}, 
q_0 \otimes \begin{bmatrix} 0\\1\\0\end{bmatrix}=\begin{bmatrix} - \frac{\sqrt{3}}{2}\\ \frac{1}{2}\\0\end{bmatrix}, 
q_0 \otimes \begin{bmatrix} 0\\0\\1\end{bmatrix}=\begin{bmatrix} 0\\  0\\ 1 \end{bmatrix}
$$



In [109]:
r0 = 0.1
l0 = 1.0
E0 = 2e11
G0 = 8e10

A0 = np.pi * r0**2
I0 = np.pi / 4 * r0**4
J0 = np.pi / 2 * r0**4

kxx =  1 * E0 * A0 / l0
kyy = 12 * E0 * I0 / l0**3
kyt =  6 * E0 * I0 / l0**2
ktx =  1 * G0 * J0 / I0
kty =  2 * E0 * I0

x0 = np.array([0.0, 0.0, 0.0])
q0 = quaternion.from_float_array([1,0,0,0])
q1 = quaternion.from_float_array([1, 0, 0, 0.000032]).normalized()
x_unit = np.array([1, 0, 0])

dx_hoge = l0 * (quaternion.rotate_vectors(q0, x_unit))

x1 = x0 + dx_hoge + np.array([0, 0.021221e-3, 0])


![matrix1](tmpabe439_thumb.png)

http://what-when-how.com/the-finite-element-method/fem-for-frames-finite-element-method-part-1/


In [110]:
def Beam_Force(x0, q0, x1, q1, l0, kxx, kyy, kyt, ktx, kty):
    q0_inv = q0.inverse()
    x1_ = quaternion.rotate_vectors(q0_inv, x1 - x0)

    dx = x1_ - np.array([l0, 0, 0])
    dq = q0_inv * q1 * 2
    
    #print(dx, dq)
    
    Fx1 = kxx * dx[0]
    Fy1 = kyy * dx[1] - kyt * dq.z
    Fz1 = kyy * dx[2] + kyt * dq.y
    
    F1  = quaternion.rotate_vectors(q0, np.array([ Fx1, Fy1, Fz1]))
    
    Tx1 = ktx * dq.x
    Ty1 = kyt * dx[2] + kty * dq.y * 2
    Tz1 =-kyt * dx[1] + kty * dq.z * 2
    T1  = quaternion.rotate_vectors(q0, np.array([ Tx1, Ty1, Tz1]))
    
    Ty0 = kyt * dx[2] + kty * dq.y
    Tz0 =-kyt * dx[1] + kty * dq.z    
    T0  = quaternion.rotate_vectors(q0, np.array([-Tx1, Ty0, Tz0]))
    
    #return F1, T1, -F1, T0
    return -F1, T0, F1, T1

F0, T0, F1, T1 = Beam_Force( x0, q0, x1, q1, l0, kxx, kyy, kyt, ktx, kty)
F0_,T0_,F1_,T1_= Beam_Force(-x1, q1,-x0, q0, l0, kxx, kyy, kyt, ktx, kty)

print(F0, F1_)
print(T0, T1_)
print(F1, F0_)
print(T1, T0_)


[  -0.         2031.79362969   -0.        ] [   -4.20445797 -2031.79390496     0.        ]
[ 0.          0.         10.58716621] [ 0.          0.         10.58716722]
[    0.         -2031.79362969     0.        ] [   4.20445797 2031.79390496   -0.        ]
[   0.            0.         2021.20646348] [   0.            0.         2021.20646449]




#### 1.1.2. Previous research

非線形3次元ビームの定式化では、ビームのデ・フォームされた形状は、ビームの変形軸と断面の回転によって記述される。したがって、ビームの構成空間は、（i）変形軸の位置ベクトルの線形空間と、（ii）断面の回転の非線形空間とからなり、非線形多様体となる。空間回転の非線形性は特別な扱いを必要とするため、3次元ビームの研究は興味深く、挑戦的なものとなっています。
In non-linear three-dimensional beam formulations, the de- formed geometry of the beam is described by the deformed axis of the beam and by the rotation of the cross-sections. The configu- ration space of the beam thus consists of (i) the linear space of the position vector of the deformed axis, and (ii) the non-linear space of rotations of cross-sections, and is thus a non-linear manifold. The non-linearity of spatial rotations requires a special treatment, which makes the study of three-dimensional beams interesting and challenging.


梁の弾性問題には中心軸からの変位と回転の評価が非常重要です。しかし、実際には回転が重要なのではなく、曲げおよびそれによって発生する歪みが重要なのであって、全体が一様に回転している場合には歪みは発生しません。一方で、ボールねじのねじ軸を弾性体として扱う場合には、離散化するべき部材の形状はは太さに対して長さが十分に短く、一次近似で十分な場合がほとんどです。そこで今回のモデルは変形が十分小さい仮定のもと、両端のノードは任意の変位および姿勢をもつものとして、その場合に発生するバネ反力について定式化する。全体の回転についてはクォータニオンに任せ、微小回転についてはクォータニオンを用いて近似を行う。



In [111]:
q0 = quaternion.from_rotation_vector(np.array([1, 0, 0]) * 0.0001)
q1 = quaternion.from_rotation_vector(np.array([0, 1, 0]) * 0.0001)

print(q1 * q0.inverse())
print(q0.inverse() * q1)


quaternion(0.9999999975, -4.99999999166667e-05, 4.99999999166667e-05, 2.49999999791667e-09)
quaternion(0.9999999975, -4.99999999166667e-05, 4.99999999166667e-05, -2.49999999791667e-09)


# ボツアイデア

$
\Delta \vec{x_R} = q_{0}^{\ast} \otimes (\vec{x_R} - \vec{x_0}) - \frac{l}{2} \begin{bmatrix} 1 \\ 0 \\ 0 \\ \end{bmatrix} \approx \begin{bmatrix} \Delta x_R \\ \Delta y_R \\ \Delta z_R \end{bmatrix} , \Delta q_R = q_0^{\ast} \otimes q_R \approx \begin{bmatrix} 1 \\ \Delta \theta_{xR} / 2 \\ \Delta \theta_{yR} / 2 \\ \Delta \theta_{zR} / 2 \\ \end{bmatrix} \\\\
\Delta \vec{x_L} = q_{0}^{\ast} \otimes (\vec{x_L} - \vec{x_0}) + \frac{l}{2} \begin{bmatrix} 1 \\ 0 \\ 0 \\ \end{bmatrix} \approx \begin{bmatrix} \Delta x_L \\ \Delta y_L \\ \Delta z_L \end{bmatrix} , \Delta q_L = q_0^{\ast} \otimes q_L \approx \begin{bmatrix} 1 \\ \Delta \theta_{xL} / 2 \\ \Delta \theta_{yL} / 2 \\ \Delta \theta_{zL} / 2 \\ \end{bmatrix} \\\\
$



$
 \begin{bmatrix} F_{xR} \\ F_{yR} \\ F_{zR} \\ T_{xR} \\ T_{yR} \\ T_{zR} \\ \end{bmatrix} =
-\begin{bmatrix} F_{x0} \\ F_{y0} \\ F_{z0} \\ T_{x0} \\ T_{y0} \\ T_{z0} \\ \end{bmatrix} =
\begin{bmatrix} 
\frac{2EA}{l} & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{96EI}{l^3} & 0 & 0 & 0 & \frac{24EI}{l^2} \\
0 & 0 & \frac{96EI}{l^3} & 0 & -\frac{24EI}{l^2} & 0 \\
0 & 0 & 0 & \frac{2GJ}{l} & 0 & 0 \\ 
0 & 0 & -\frac{24EI}{l^2} & 0 & \frac{8EI}{l} & 0 \\
0 & \frac{24EI}{l^2} & 0 & 0 & 0 & \frac{8EI}{l} \\
\end{bmatrix}
\begin{bmatrix} \Delta x_R \\ \Delta y_R \\ \Delta z_R \\ \Delta \theta_{xR} \\ \Delta \theta_{yR} \\ \Delta \theta_{zR} \\ \end{bmatrix} \\\\
$


$
\begin{align}\begin{cases}
\Delta x_R &= -\Delta x_L \\
\Delta \theta_{xR} &= -\Delta \theta_{xL} \\
\end{cases}\end{align}
$

$
\begin{align}\begin{cases}
4 \Delta y_R + l \Delta \theta_{zR} &=  4 \Delta y_L - l \Delta \theta_{zL} \\
3 \Delta y_R - l \Delta \theta_{zR} &=  3 \Delta y_L - l \Delta \theta_{zL} \\
\end{cases}\end{align}
$

$
\begin{align}\begin{cases}
4 \Delta z_R - l \Delta \theta_{yR} &= -4 \Delta z_L - l \Delta \theta_{yL} \\
3 \Delta z_R - l \Delta \theta_{yR} &=  3 \Delta z_L + l \Delta \theta_{yL} \\
\end{cases}\end{align}
$


In [112]:
q0 = quaternion.from_rotation_vector(np.array([1, 0, 0]) * 0.001)
q1 = quaternion.from_rotation_vector(np.array([0, 1, 0]) * 0.001)
q2 = quaternion.from_rotation_vector(np.array([0, 0, 1]) *-0.001)

print(q2 * q1 * q0)


quaternion(0.999999624875055, 0.000500249854114599, 0.000499749854218766, -0.000500249854114599)


In [113]:
q0 / q1


quaternion(0.999999750000021, 0.000499999916666671, -0.000499999916666671, -2.49999979166667e-07)

In [114]:
eye3 = np.eye(3)
quaternion.rotate_vectors(q0.inverse(), eye3)

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  9.99999500e-01, -9.99999833e-04],
       [ 0.00000000e+00,  9.99999833e-04,  9.99999500e-01]])

In [115]:
hoge = np.array([[-1, 1], [ 1, 1]])
fuga = np.array([[ 2, 2], [-3, 3]])

hoge @ fuga

24 * np.linalg.inv(hoge @ fuga)

array([[-5.,  1.],
       [-1.,  5.]])

Two types of modeling of beams by quaternions have been presented by the same laboratory.

|Kinematically Exact Beam Finite Elements Based on Quaternion Algebra (2014)|A consistent strain-based beam element with quaternion representation of rotations (2020)|
|:---:|:---:|
|![1‗2014](1_2014.png)|![1_2020](1_2020.png)|
|Eva Zupan, Miran Saje, **Dejan Zupan**|Damjan Loli, **Dejan Zupan**, Miha Brojan|
|University of Ljubljana|University of Ljubljana|
|Simple|Complex|
|displacement-based|strain-based|
|12 DOFs per element|26 DOFs per element|
|**to small shear deformations**|to larger shear deformations|

In order to deal with the deformation of the ballscrew this time, we do not need to consider "larger shear deformations", so I will deal with the **2014 model**.




#### Beam Model

input:

$
p=0...N+1,
\chi_p,
r_g(\chi_p),
\hat{q}(\chi_p)
$

parameter:

$
\mathscr C _N = \begin{bmatrix} EA_1 & 0 & 0 \\ 0 & GA_2 & 0 \\ 0 & 0 & GA_3 \end{bmatrix} \gamma _G(\chi _k),
\mathscr C _M = \begin{bmatrix} GJ_1 & 0 & 0 \\ 0 & EJ_2 & 0 \\ 0 & 0 & EJ_3 \end{bmatrix} \kappa _G(\chi _k)
$

local variable:

$
k=1...N,
n_g(\chi_k),
\hat{m}_g(\chi_k),
N_g(\frac{L}{2}),
M_g(\frac{L}{2})
$

$
r'_g(\chi) = \sum_{p=0}^{N+1} L'(\chi_p)r_g(\chi_p)
$

$
\gamma _G(\chi _k) = \left[ \hat{q}^*(\chi_k) \circ \hat r' _g(\chi _k) \circ \hat{q}(\chi_k) \right] _{\mathbb{R}^3},
\kappa _G(\chi _k) = 2 \left[ \hat{q}^*(\chi_k) \circ \hat{q}'(\chi_k) \right] _{\mathbb{R}^3}
$

output (k):

$
\left[\hat{q}(\chi_k) \circ \mathscr {\hat C} _N \circ \hat{q}^*(\chi_k) \right]' _{\mathbb{R}^3} + n_g(\chi_k) = 0
$

$
\left( \hat{q}(\chi_k) \circ \mathscr {\hat C} _M \circ \hat{q}^*(\chi_k) \right)' + \hat{m}_g(\chi_k) - N_g(\frac{L}{2})\times r'_g(\chi_k)  + \int_{\chi_k}^{L / 2} n_g(\chi) \,d\chi \times r'_g(\chi_k) = 0
$

output:

$
F_g(0) + N_g(\frac{L}{2}) + \int_{0}^{L / 2} n_g(\chi) \,d\chi = 0
$

$
P_g(0) + M_g(\frac{L}{2}) - N_g(\frac{L}{2}) \times \left( r_g(\frac{L}{2}) - r_g(0) \right) - \int_{0}^{L / 2} n_g(\chi) \times \left(r_g(\chi) - r_g(0) \right) \,d\chi + \int_{0}^{L / 2} m_g(\chi) \,d\chi = 0
$

$
F_g(L) - N_g(\frac{L}{2}) + \int_{L / 2}^{L} n_g(\chi) \,d\chi = 0
$

$
P_g(L) - M_g(\frac{L}{2}) - N_g(\frac{L}{2}) \times \left( r_g(L) - r_g(\frac{L}{2}) \right) + \int_{L / 2}^{L} n_g(\chi) \times \left(r_g(L) - r_g(\chi) \right) \,d\chi + \int_{L / 2}^{L} m_g(\chi) \,d\chi = 0
$


In [116]:
def CN_CM_Rectangle(E, G, L, t, h):
    
    A1 = t * h
    A2 = L * t
    A3 = h * L
    
    CN = np.array([E * A1, G * A2, G * A3])

    J_Rectangle = lambda b, d: b * d * (b*b + d*d) / 12
    
    J1 = J_Rectangle(t, h)
    J2 = J_Rectangle(L, t)
    J3 = J_Rectangle(h, L)
    
    CM = np.array([G * J1, E * J2, E * J3])
    
    return CN, CM


In [117]:
E = 1e7
G = 1e13
L = 1
t = 0.1
h = 0.1 # 0.1~10

CN, CM = CN_CM_Rectangle(E, G, L, t, h)

print(CN, CM)


[1.e+05 1.e+12 1.e+12] [1.66666667e+08 8.41666667e+04 8.41666667e+04]


#### hoge

$\delta \hat \kappa _G = 2 \delta \hat q ^* \circ \hat q' + 2 \hat q ^* \circ \delta \hat q'$

In [118]:
def calc_delta_kappaG(q0, q1):
    return 1

$
y ^{[n + 1]} = y ^{[n]} + \delta y
$

$
r_g ^{p [n + 1]} = r_g ^{p [n]} + \delta r ^p _g \qquad(p =0,..., N+1)
$

$
N_g(\chi) = N_g(\frac{L}{2}) + \int_{\chi}^{L / 2} n_g \,d\xi
$

$
M_g(\chi) = M_g(\frac{L}{2}) + \int_{\chi}^{L / 2} m_g \,d\xi - \int_{\chi}^{L / 2} N_g \times r'_g \,d\xi
$

$
r'_g(\chi_k) \approx \frac{r_g(\chi_{k + 1}) - r_g(\chi_{k - 1})}{2} \frac{L}{N + 1} \\\\
r'_g(\chi_k) \approx \frac{r_g(\chi_{k + 1}) - r_g(\chi_{k - 1})}{2} \frac{L}{N + 1}
$


